# Final Project
Tamir Shoresh

Dan Goren

Liora Cohen

# Introduction
1. Most clustering methods have been developed under the assumption of a static database. In modern applications, the database is usually updated in real-time. Shkapsky et al, 2016, deals with the issues arising from utilizing known clustering methods on real-time data. 

2. Algorithms that were originally designed for a sequential nature, must often be redesigned to effectively use the distributed computational resources. Iaani et al, 2018, presents the CLUBS+ algorithm for efficient parallel clustering.

3. Bisecting k-means is a hierarchal k-means algorithm. The bisecting steps of clusters on the same level are grouped together to increase parallelism (Abirami et al, 2016).


# Dataset

The Customer Classification dataset is a set of customer data including:
   1. region of store
   2. tenure
   3. age
   4. income
   5. marital status
   6. address
   7. education group
   8. employment years
   9. retirement status
   10. gender
   11. residential area code
   12. customer catagory (target variable)
    

# Methods
In this work, we wish to create a Kafka server to mimic the properties of real-time data, just as in Shkapsky et al, 2016. This will be done by creating a producer that reads from a csv file 100 lines and publishes them to the topic "data". we will also add a 5 second sleep between publishes. On the consumer side, we will read the data from the topic into a Spark RDD.
Next, we would like to compare 2 clustering methods. The first is k-means, which does not utilize spark in an efficient maner. The second method is bisecting k-means, which utilizes spark more efficiently, but due to its algorithm may have less accuracy. we will compare them both on calculation time and on accuracy.


In [1]:
# creating the topic "data" in port 9092
!/usr/local/kafka/kafka_2.13-3.2.1/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic data

Created topic data.


In [ ]:
# imports
from confluent_kafka import Producer
import time
import csv
import json
import numpy as np
# create producer
p = Producer({'bootstrap.servers': 'localhost:9092'})
# open csv file
i=0
with open('train.csv') as f:
    row_count = 0
    for _ in f:
        row_count +=1
# run producer in infinite loop
while True: 
    flag = 0
    # create dictionary of features that contains words as keys and numbers as values
    d={}
    if flag == 0 :
        # Open file 
        file = open('train.csv')
        reader_obj = csv.reader(file)
        
        for row in reader_obj:
            if i<row_count+1:
                # adding a row to the dictionary
                d[i] = row
                i += 1                
                # sleep for 5 seconds
                if i % 100 == 0:
                    print(row)
                    # every 100 lines (customers), encode the dictionary (for publishing) and send it through the producer
                    d_encoded = json.dumps(d).encode('utf-8')
                    p.produce('data', value=d_encoded)
                    time.sleep(5)
                    # reset dictionary
                    d={}
                    print("Finished " + str(i))
            else: 
                flag = 1
                break     
            
            
            


%4|1675706926.377|TERMINATE|rdkafka#producer-4| [thrd:app]: Producer terminating with 15 messages (200229 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


['657', '0', '2.5', '0', '0', '0', '37', '0.7', '141', '2', '1', '476', '961', '1412', '14', '3', '18', '0', '1', '0', '0']
Finished 100
['609', '0', '0.5', '0', '3', '0', '26', '0.3', '93', '4', '4', '938', '1948', '1866', '11', '10', '14', '1', '1', '1', '1']
Finished 200
['928', '1', '0.5', '1', '11', '0', '56', '0.7', '80', '8', '13', '221', '1243', '666', '19', '10', '5', '0', '1', '0', '0']
Finished 300
['1323', '1', '2.5', '1', '10', '1', '28', '0.2', '131', '4', '12', '162', '619', '1892', '10', '0', '15', '1', '1', '1', '1']
Finished 400
['1926', '1', '1.7', '1', '1', '1', '33', '0.4', '172', '3', '2', '982', '1157', '2488', '6', '1', '14', '1', '0', '0', '2']
Finished 500
['1137', '0', '2.2', '0', '6', '1', '3', '0.9', '173', '5', '7', '1250', '1285', '316', '15', '14', '17', '1', '0', '0', '0']
Finished 600
['1940', '1', '0.9', '1', '4', '0', '17', '0.7', '93', '5', '7', '112', '858', '2297', '14', '6', '8', '0', '1', '0', '2']
Finished 700
['1800', '1', '0.7', '0', '2', '1'

In [5]:
p.flush()

0